# ALC Simulations based on Analytical Solutions

### $\Delta_1$ resonance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.constants import pi
import seaborn as sns
from plot_settings import set_demonlab_style

def muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel):
    """
    Calculate the time-integrated muon polarization for a given magnetic field and angle theta.
    """
    gamma_muon = 135.5 # in MHz/T
    inv_muon_lifetime = 0.4551 # in MHz
    osc_relaxation = inv_muon_lifetime + off_resonance_state_rate # in MHz

    def calc_q(theta, D_parallel):
        return 0.75 * D_parallel * np.sin(theta) * np.cos(theta)

    def calc_nu_mu(A_iso, D_parallel, theta):
        return (A_iso + D_parallel/2 * (3 * np.cos(theta)**2 - 1)) / 2

    nominator = 0.5 * calc_q(theta, D_parallel)**2 * Pz_0
    denominators = [(osc_relaxation/2/pi)**2 + calc_q(theta, D_parallel)**2 + (gamma_muon*magnetic_field - calc_nu_mu(A_iso, D_parallel, theta))**2 for magnetic_field in magnetic_fields]

    return [1 - nominator/denominator for denominator in denominators]

In [ ]:
magnetic_fields = np.linspace(1.8675, 1.9325, 3000) # in Tesla
thetas = np.radians([1, 5, 20, 45, 70, 85, 89]) # given in degrees and converted to radians
Pz_0 = 1.0
off_resonance_state_rate = 0
A_iso = 514.8 # in MHz
D_parallel = 2 # in MHz

df = pd.DataFrame({'B / T': magnetic_fields})
for theta in thetas:
    df[f"θ = {theta:.2f} rad"] = muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)

polarizations = muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)

# Melt for seaborn plotting
df_melted = df.melt(id_vars="B / T", var_name="θ", value_name="Pz")

# Plot
plt.figure(figsize=(8,5))
sns.lineplot(data=df_melted, x="B / T", y="Pz", hue="θ")
plt.xlabel("B / T")
plt.ylabel(r"$P_z$")
plt.legend(title="Angle")
